<a href="https://colab.research.google.com/github/edgarmariorivera-searce/llm/blob/main/%5BJT%5D_vertex_llm_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapting [this example code](https://docs.google.com/document/d/1zY2x607QxoBdRwL_x9tmgxKFIUZRdvEiIZx6w-fgxWA/edit) from OCTO

In [ ]:
# @title Authenticate
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
# @title Install Python Libraries
!pip install -q  chromadb
!pip install -q  langchain
!pip install -q  google-cloud-core

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00


In [ ]:
! gsutil cp gs://vertex_sdk_llm_private_releases/SDK/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl .

In [ ]:
# @title Install Vertex AI SDK (restart runtime when completed)
# !gsutil cp gs://agravat-octo-aif-sandbox-central/llm_demo/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl .
! gsutil -q cp gs://vertex_sdk_llm_private_releases/SDK/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl .

!pip install -q google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl "shapely<2.0.0"
!pip install --upgrade --user -q google-cloud-storage
!pip install -q gcsfs

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Copying gs://vertex_sdk_llm_private_releases/SDK/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl...
/ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.0 MB/s eta 0:00:00


{'status': 'ok', 'restart': True}

In [ ]:
PROJECT_ID='cpg-cdp'
LOCATION = 'us-central1'
! gcloud config set project $PROJECT_ID

Updated property [core/project].


## define LLM classes and instantiate

In [ ]:
#@title Define LLM classes and instantiate
import time

from google.cloud.aiplatform.private_preview import language_models

from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM


def rate_limit(max_per_minute):
  period = 60 / max_per_minute
  while True:
    before = time.time()
    yield
    after = time.time()
    elapsed = after - before
    sleep_time = max(0, period - elapsed)
    if sleep_time > 0:
      print(f'Sleeping {sleep_time:.1f} seconds')
      time.sleep(sleep_time)


class VertexEmbeddings(Embeddings):

  def __init__(self, model, *, requests_per_minute=15):
    self.model = model
    self.requests_per_minute = requests_per_minute

  def embed_documents(self, texts):
    limiter = rate_limit(self.requests_per_minute)
    results = []
    docs = list(texts)

    while docs:
      # Working in batches of 2 because the API apparently won't let
      # us send more than 2 documents per request to get embeddings.
      head, docs = docs[:2], docs[2:]
      chunk = self.model.get_embeddings(head)
      results.extend(chunk)
      next(limiter)

    return [r.values for r in results]

  def embed_query(self, text):
    single_result = self.embed_documents([text])
    return single_result[0]


class VertexLLM(LLM):

  model: language_models.Any
  predict_kwargs: dict

  def __init__(self, model, **predict_kwargs):
    super().__init__(model=model, predict_kwargs=predict_kwargs)

  @property
  def _llm_type(self):
    return 'vertex'

  def _call(self, prompt, stop=None):
    result = self.model.predict(prompt, **self.predict_kwargs)
    return str(result)

  @property
  def _identifying_params(self):
    return {}

# NOTE: Use staging to get 100qps max throughput for embedding indexing
# The embedding content is the same as production so you can use staging
# for indexing and production for querying if desired.
#language_models.TextEmbeddingModel._LLM_ENDPOINT_NAME = (
#  'projects/678515165750/locations/us-central1/endpoints/8156038716377268224')

REQUESTS_PER_MINUTE = 15
REQUESTS_PER_MINUTE = 6000

model = language_models.TextGenerationModel.from_pretrained('text-bison-001')
llm = VertexLLM(
  model,
  max_output_tokens=256,
  temperature=0.1,
  top_p=0.8,
  top_k=40
)
embedding = VertexEmbeddings(language_models.TextEmbeddingModel(), requests_per_minute=REQUESTS_PER_MINUTE)

llm

VertexLLM(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f667cab94f0>, model=<google.cloud.aiplatform.private_preview.language_models.language_models.TextGenerationModel object at 0x7f66ac455a90>, predict_kwargs={'max_output_tokens': 256, 'temperature': 0.1, 'top_p': 0.8, 'top_k': 40})

## mount google drive

In [ ]:
#@title Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

# persistent_path = '/content/drive/Shareddrives/AIF LLM Demos/BigQuery'

# persistent_path = '/content/drive/Colab Notebooks/llm-demos'
persistent_path = '/content/drive/My Drive/Colab Notebooks/llm-demos'

## BigQuery setup

In [ ]:
# @title BigQuery Setup
%load_ext google.cloud.bigquery

from google.cloud import bigquery
from google.colab import data_table

# Project ID inserted based on the query results selected to explore
# project = 'octo-aif-sandbox'
project = 'cpg-cdp'

# Location inserted based on the query results selected to explore
location = 'US'

client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Use Case 1: Summarization

In [ ]:
from langchain.document_loaders.dataframe import DataFrameLoader
from langchain.indexes import VectorstoreIndexCreator

# The job has already been executed so we don't need to rerun the query
job_id = 'bquxjob_20b2259d_18738815f10'

job = client.get_job(job_id)
print(f'SQL: {job.query}')

df = job.to_dataframe()

loader = DataFrameLoader(df, page_content_column="body")
bbc_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
  'persist_directory': f'{persistent_path}/bbc'
}).from_loaders([loader])

Forbidden: ignored

#### We can then summarize relevant articles based on similarity search

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set up some parameters
SEARCH_RESULT_COUNT = 5
SIMILIARITY_THRESHOLD = 0.7
question = (
    'As a commodity trader with business operations in the Europe, '
    'I want to know if there are regional events '
    'that are driving commodity prices up?'
)

# Based on the question, sarch for relevant articles
similar_docs = bbc_index.vectorstore.similarity_search_with_score(question, llm=llm, k=SEARCH_RESULT_COUNT)
filtered_docs = list(filter(lambda doc: doc[1] <= SIMILIARITY_THRESHOLD, similar_docs))

# Merge the resulting articles, chunk them up and ask Vertex LLM to summarize
merged = "\n".join([doc.page_content for doc, score in filtered_docs])
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1000,
  chunk_overlap  = 200,
  length_function = len,
)
docs = text_splitter.create_documents([merged])
prompt_template = """Write a concise summary of the following text in bullet points:

{text}

CONCISE SUMMARY IN BULLET POINTS:"""
summary_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=summary_prompt)
context = chain.run(docs)
print(context)

In [ ]:
for doc, score in filtered_docs:
  print(f"Title: {doc.metadata['title']}, Score: {score} ")
  print("")

## Use Case 2: Question and Answer

#### Run a query against the [Google Cloud Release Note dataset](https://pantheon.corp.google.com/marketplace/product/bigquery-public-datasets/google_cloud_release_notes?e=13802955&jsmode=O&mods=-autopush_coliseum&project=octo-aif-sandbox)

In [ ]:
%%bigquery release_notes_df --project $project


SELECT description, product_name, published_at, product_version_name, release_note_type
FROM `bigquery-public-data.google_cloud_release_notes.release_notes`
ORDER BY published_at DESC
LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

### clean up data for indexing

In [ ]:
release_notes_df['published_at'] = release_notes_df['published_at'].astype(str)
release_notes_df = release_notes_df.fillna("")

In [ ]:
df_loader = DataFrameLoader(release_notes_df, page_content_column="description")

release_note_index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_kwargs={
        'persist_directory': f'{persistent_path}/release_notes'
    }
).from_loaders([df_loader])

In [ ]:
from langchain.chains import RetrievalQA, ConversationChain, SimpleSequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

question = "What are the new features in BigQuery?"

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=release_note_index.vectorstore.as_retriever()
)

retrieval_chain({"query": question}, return_only_outputs=True)

Exception ignored in: <function PersistentDuckDB.__del__ at 0x7f6665a471f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/chromadb/db/duckdb.py", line 446, in __del__
    self.persist()
  File "/usr/local/lib/python3.9/dist-packages/chromadb/db/duckdb.py", line 393, in persist
    os.makedirs(self._save_folder)
  File "/usr/lib/python3.9/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.9/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.9/os.py", line 225, in makedirs
    mkdir(name, mode)
OSError: [Errno 95] Operation not supported: '/content/drive/Shareddrives/AIF LLM Demos'
Exception ignored in: <function PersistentDuckDB.__del__ at 0x7f6665a471f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/chromadb/db/duckdb.py", line 446, in __del__
    self.persist()
  File "/usr/local/lib/python3.9/dist-packages/chromadb/db/duckdb.py", 

Token indices sequence length is longer than the specified maximum sequence length for this model (1534 > 1024). Running this sequence through the model will result in indexing errors


{'result': 'Non-incremental materialized views'}

## Use Case 3: Tabular data with numeric value
We will use the Google DEI dataset for this experiment. In this case, we want to convert structured data into unstructured text so that we can feed it into Vertex LLM for Q&A. Using the table as context, we can ask the Vertex LLM to generate a sentence to describe each row, which we will use for the conversion.

In [ ]:
%%bigquery dei_df --project $project

SELECT * FROM `bigquery-public-data.google_dei.dar_intersectional_representation`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
dei_df

,workforce,report_year,gender_us,race_asian,race_black,race_hispanic_latinx,race_native_american,race_white
0,overall,2014,women,0.100,0.011,0.015,0.005,0.176
1,overall,2015,women,0.105,0.010,0.016,0.004,0.171
2,overall,2016,women,0.110,0.011,0.017,0.004,0.165
3,overall,2017,women,0.118,0.012,0.017,0.003,0.159
4,overall,2018,women,0.125,0.012,0.017,0.003,0.155
...,...,...,...,...,...,...,...,...
67,leadership,2018,men,0.200,0.011,0.019,0.004,0.524
68,leadership,2019,men,0.209,0.015,0.020,0.003,0.501
69,leadership,2020,men,0.211,0.015,0.022,0.002,0.491
70,leadership,2021,men,0.207,0.018,0.025,NaN,0.477


Let's see if we can ask Vertex LLM to generate a sentence that describe one of the rows, which we will then use as a seed to generalize the translation process from tabular data to natural language text

In [ ]:
# Limit the size of the table due to context window
dei_table = dei_df.head(5).to_markdown()

prompt = f"""
  Use the following table in markdown format to answer the question at the end.

  {dei_table}

  Generate a sentence for each row to describe the meaning of the values
"""
response = model.predict(prompt)
response

The table shows the percentage of women in the workforce in the United States by race and ethnicity from 2014 to 2018.

In 2014, 10% of women in the workforce were Asian, 1.1% were Black, 1.5% were Hispanic or Latino, 0.5% were Native American, and 76.4% were White.

In 2015, 10.5% of women in the workforce were Asian, 1.0% were Black, 1.6% were Hispanic or

Using the generated sentence as an example, convert the structured table into natural language text with one sentence per row

In [ ]:
import pandas as pd

data = []
for index, rows in dei_df.iterrows():

  total_percent = rows['race_asian'] + rows['race_black'] + rows['race_hispanic_latinx'] + rows['race_native_american'] + rows['race_white']

  text = """
  In year {0}, {1} made up {2}% of the {3} workforce, and the racial makeup of the workforce was {4}% Asian,
  {5}% Black, {6}% Hispanic or Latino, {7}% Native American, and {8}% White.
  """.format(
      rows['report_year'],
      rows['gender_us'],
      round(total_percent, 2)*100,
      rows['workforce'],
      round(rows['race_asian']*100, 2),
      round(rows['race_black']*100, 2),
      round(rows['race_hispanic_latinx']*100, 2),
      round(rows['race_native_american']*100, 2),
      round(rows['race_white']*100, 2))

  data.append(text)

# Put it in a dataframe so we can easily index it
nl_dei_df = pd.DataFrame(data, columns=['text'])

In [ ]:
nl_dei_df

,text
0,"\n In year 2014, women made up 31.0% of the o..."
1,"\n In year 2015, women made up 31.0% of the o..."
2,"\n In year 2016, women made up 31.0% of the o..."
3,"\n In year 2017, women made up 31.0% of the o..."
4,"\n In year 2018, women made up 31.0% of the o..."
...,...
67,"\n In year 2018, men made up 76.0% of the lea..."
68,"\n In year 2019, men made up 75.0% of the lea..."
69,"\n In year 2020, men made up 74.0% of the lea..."
70,"\n In year 2021, men made up nan% of the lead..."


Let's index the sentences

In [ ]:
df_loader = DataFrameLoader(nl_dei_df, page_content_column="text")

dei_index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_kwargs={
        'persist_directory': f'{persistent_path}/dei'
    }
).from_loaders([df_loader])

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

def ask_question(question, max_results=5, threshold=0.5):
  '''
  Based on the question, sarch for relevant articles
  '''

  similar_docs = dei_index.vectorstore.similarity_search_with_score(question, llm=llm, k=max_results)
  filtered_docs = list(filter(lambda doc: doc[1] <= threshold, similar_docs))
  context = "\n".join([doc.page_content for doc, score in filtered_docs])

  prompt = f"""
  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

  {context}

  Question: {question}
  Answer:
  """
  return model.predict(prompt)

Let's ask the LLM to retrieve exact values from the dataset and see if they are correct (hint: correct)

In [ ]:
ask_question('What percentage of the male workers are Asian in 2016')

22.9%

The question is "What percentage of the male workers are Asian in 2016".

In year 2016, men made up 73.0% of the overall workforce, and the racial makeup of the workforce was 22.9% Asian.

So the answer is 22.9%.

Let's see if we can consistenly get the right answer (hint: incorrect)

In [ ]:
ask_question('What is the percentage of Asian male tech worker in 2016')

55.5%

To answer this question, we need to know the percentage of men in the tech workforce in 2016 and the percentage of Asians in the tech workforce in 2016. The percentage of men in the tech workforce in 2016 is 83.0%, and the percentage of Asians in the tech workforce in 2016 is 28.0%. So the percentage of Asian male tech worker in 2016 is 83.0% * 28.0% = 23.64%.

And here is the actual value from the table

In [ ]:
%%bigquery --project $project

SELECT race_asian, workforce
FROM `bigquery-public-data.google_dei.dar_intersectional_representation`
WHERE
  report_year = 2016 AND
  gender_us = 'men'

Query is running:   0%|          |

Downloading:   0%|          |

,race_asian,workforce
0,0.229,overall
1,0.280,tech
2,0.109,non_tech
3,0.184,leadership


It was quite fiddly and it's obvious not always giving us the right answer. But that could be because each sentence packs a lot of data!

Let's try to reduce the amount of information within each sentence so that semantic search has a better chance of picking up the right info. For example, instead of encoding all the fields in each row into the same sentence, we serialize each row into multiple sentences (e.g. In 2015, Asian male made up X% of the tech workforce)

In [ ]:
def get_sentence(year, gender, race, workforce, percent):
  return f'In {year}, {race} {gender} made up {percent}% of the {workforce} workforce'

data = []
for index, rows in dei_df.iterrows():
  for race in ['asian', 'black', 'hispanic latinx', 'native american', 'white']:
    # Special case
    key = 'race_' + race.replace(' ', '_')
    # Get the percentage of the workforce in the
    text = get_sentence(
        rows['report_year'],
        rows['gender_us'],
        race,
        rows['workforce'],
        round(rows[key]*100, 2))
    data.append(text)

# Put it in a dataframe so we can easily index it
nl_dei_df_v2 = pd.DataFrame(data, columns=['text'])

df_loader = DataFrameLoader(nl_dei_df_v2, page_content_column="text")
dei_index = VectorstoreIndexCreator(embedding=embedding, vectorstore_kwargs={
  'persist_directory': f'{persistent_path}/dei_v2'
}).from_loaders([df_loader])

In [ ]:
ask_question('What is the percentage of Asian male tech worker in 2016')

28.0%.

In [ ]:
ask_question('Are there more black female leaders in 2022?')

> Yes, there are more black female leaders in 2022 than in any other year.

The context states that black women made up 2.0% of the leadership workforce in 2022, which is higher than the percentage of black women in the leadership workforce in any other year. Therefore, we can conclude that there are more black female leaders in 2022 than in any other year.

In [ ]:
%%bigquery --project $project

SELECT report_year, race_black FROM `bigquery-public-data.google_dei.dar_intersectional_representation` WHERE gender_us = 'women' AND workforce = 'leadership'

Query is running:   0%|          |

Downloading:   0%|          |

,report_year,race_black
0,2014,0.010
1,2015,0.012
2,2016,0.010
3,2017,0.011
4,2018,0.013
5,2019,0.011
6,2020,0.011
7,2021,0.013
8,2022,0.020


In [ ]:
ask_question('Overall, which ethnicity has the highest representation in the workforce for 2022?')

2022, white men made up 33.2% of the overall workforce

In [ ]:
%%bigquery --project $project

SELECT * FROM `bigquery-public-data.google_dei.dar_intersectional_representation` WHERE report_year = 2022 AND workforce = 'overall'

Query is running:   0%|          |

Downloading:   0%|          |

,workforce,report_year,gender_us,race_asian,race_black,race_hispanic_latinx,race_native_american,race_white
0,overall,2022,women,0.153,0.023,0.024,0.003,0.151
1,overall,2022,men,0.279,0.030,0.046,0.005,0.332
